In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers,Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report


In [2]:
batch_size=3000
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
train_dir='/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train'
# Load the training data
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(64, 64),  batch_size=batch_size, class_mode='binary')
test_dir='/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test'                                                  
# Load the test data
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(64,64),batch_size=batch_size, class_mode='binary')



Found 100000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


In [3]:
train_generator = train_generator.next()
train_samples = train_generator[0]
train_labels = train_generator[1]


In [4]:
#feature extraction
from tensorflow.keras.applications import VGG16
pretrained_model = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
# freeze
for layer in pretrained_model.layers:
    layer.trainable = False


58889256/58889256 [==============================] - 3s 0us/step


In [5]:
from tensorflow.keras.layers import GlobalAveragePooling2D
x = GlobalAveragePooling2D()(pretrained_model.output)
feature_extractor = Model(inputs=pretrained_model.input, outputs=x)

In [6]:
train_features = feature_extractor.predict(train_samples)
print(train_features.shape)


94/94 [==============================] - 58s 611ms/step
(3000, 512)


In [7]:
# Define the parameters
population_size = 100
num_features = train_features.shape[1]
num_generations = 50
mutation_rate = 0.01
# subset_size = 300
early_stop = 10
train_labels=train_generator[1]
train_features=train_features


In [8]:
#genatic algorism

def evaluate_fitness(features):
    selected_features = train_features[:, features]
    X_train, X_val, y_train, y_val = train_test_split(selected_features, train_labels, test_size=0.2)
    classifier = SVC()
    classifier.fit(X_train, y_train)
    accuracy = classifier.score(X_val, y_val)
    return accuracy


def crossover(parent1, parent2):
    crossover_point = np.random.randint(1, num_features)
    child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    return child

def mutate(individual):
    for i in range(num_features):
        if np.random.rand() < mutation_rate:
            individual[i] = 1 - individual[i]
    return individual


population = np.random.randint(0, 2, size=(population_size, num_features))
best_fitness = 0
count = 0
for generation in range(num_generations):
    fitness_scores = np.array([evaluate_fitness(features) for features in population])

    parents = population[np.random.choice(population_size, size=population_size, replace=True, p=fitness_scores/np.sum(fitness_scores))]

    offspring = []
    for i in range(population_size):
        parent1 = parents[i]
        parent2 = parents[np.random.randint(population_size)]
        child = crossover(parent1, parent2)
        child = mutate(child)
        offspring.append(child)
    population = np.array(offspring)

    best_individual = population[np.argmax(fitness_scores)]
    current_fitness = evaluate_fitness(best_individual)
    if current_fitness > best_fitness:
        best_fitness = current_fitness
        count = 0
    else:
        count += 1
        if count >= early_stop:
            break

selected_features = train_features[:, best_individual == 1]
num_selected_features = np.sum(best_individual == 1)
print("number of selected features:", num_selected_features)



number of selected features: 242


In [9]:
X_train, X_test, y_train, y_test = train_test_split(selected_features, train_labels, test_size=0.4)
classifier = SVC()
classifier.fit(X_train, y_train)
y_pred=classifier.predict(X_test)


In [10]:
accuracy = accuracy_score(y_pred,y_test)
print("Accuracy: {:.4f}".format(accuracy))
conf_matrix = confusion_matrix(y_pred,y_test)
print("Confusion Matrix:")
print(conf_matrix)
report=classification_report(y_pred,y_test)
print('classification_report')
print(report)

Accuracy: 0.7992
Confusion Matrix:
[[477 103]
 [138 482]]
classification_report
              precision    recall  f1-score   support

         0.0       0.78      0.82      0.80       580
         1.0       0.82      0.78      0.80       620

    accuracy                           0.80      1200
   macro avg       0.80      0.80      0.80      1200
weighted avg       0.80      0.80      0.80      1200

